In [1]:
import torch.nn as nn
import torch.nn.functional as F
import torch
import numpy as np
import math


In [17]:
L = 4. #length
N = 1. #batch
E = 512 # dimension
input = torch.randn([L, N, E])

In [18]:
input.size()

torch.Size([4, 1, 512])

In [19]:
qkv_layer = nn.Linear(E, 3*E)

In [20]:
qkv = qkv_layer(input)

In [21]:
qkv.shape

torch.Size([4, 1, 1536])

In [22]:
head_num = 8
head_dim = E // head_num

In [23]:
qkv = qkv.reshape(L, N, head_num, 3 * head_dim)

In [24]:
qkv.shape

torch.Size([4, 1, 8, 192])

In [25]:
qkv = qkv.permute(2,1,0,3) #[num_head, batch_num, length, 3*head_dim]
qkv.shape

torch.Size([8, 1, 4, 192])

In [26]:
q, k, v = qkv.chunk(3, dim = -1)

In [28]:
def attention(query, key, value, mask = None):
    d_k = query.size(-1)
    scaled = torch.matmul(query, key.transpose(-2,-1))/math.sqrt(d_k)
    attention = F.softmax(scaled, dim = -1)
    return torch.matmul(attention, value), attention

In [29]:
values, attenion = attention(q, k, v)

In [30]:
values.shape

torch.Size([8, 1, 4, 64])

In [32]:
values = values.reshape(L, N, head_num * head_dim)

In [33]:
values.shape

torch.Size([4, 1, 512])

In [35]:
final_linear_layer = nn.Linear(E,E)
out = final_linear_layer(values)

In [37]:
out.shape

torch.Size([4, 1, 512])

In [95]:
def multi_head_self_attention(input, num_heads, attn_w, output_w):

    """
    TODO: Your implementation for the multihead attention function.
    We assume the input and the output have the same shape.
    input: the input with shape [L, N, E],
         L is the sequence length,
         N is the batch size,
         E is the embedding dimension.

    num_heads: number of the attention heads each with dimension E // num_heads.

    attn_w: the weight for the query, key, and value, with shape [3 * E, E].
    output_w: the additional linear layer with shape [E, E].
    """
    length = input.size(0)#L
    print(f"length {length}")
    batch_size = input.size(1)# N
    print(f"batch {batch_size}")
    emb_dim = input.size(-1) # E
    print(f"emb_dim {emb_dim}")
    d_k = emb_dim//num_heads
    print(f"head_dim {d_k}")

    q_layer = nn.Linear(emb_dim,  emb_dim)
    k_layer = nn.Linear(emb_dim,  emb_dim)
    v_layer = nn.Linear(emb_dim,  emb_dim)
    q = q_layer(input)
    k = k_layer(input)
    v = v_layer(input)
    qkv = torch.cat((q,k,v), dim = -1) # shape[4,1,12]
    print(qkv.shape)
    #qkv = torch.matmul(qkv, attn_w) # shape [4,1,4]
    qkv = qkv.reshape(length, batch_size, num_heads, 3*d_k)
    print(qkv.shape)
    qkv = qkv.permute(2,1,0,3)  #[num_head, batch_num, length, 3*head_dim]
    q, k, v = qkv.chunk(3, dim = -1)

    print(f"q shape {q.shape}")
    print(f"k shape {k.shape}")
    print(f"v shape {v.shape}")
    '''
    q = torch.matmul(q, attn_w[:emb_dim, :emb_dim])
    k = torch.matmul(k, attn_w[emb_dim:(emb_dim*2), :emb_dim])
    v = torch.matmul(v, attn_w[(emb_dim*2):, :emb_dim])'''


    values, attenion = attention(q, k, v)
    values = values.reshape(length, batch_size, num_heads * d_k)
    final_linear_layer = nn.Linear(emb_dim, emb_dim)
    out = torch.matmul(final_linear_layer(values), output_w)
    return out



In [115]:
def multi_head_self_attention(input, num_heads, attn_w, output_w):

    """
    TODO: Your implementation for the multihead attention function.
    We assume the input and the output have the same shape.
    input: the input with shape [L, N, E],
         L is the sequence length,
         N is the batch size,
         E is the embedding dimension.

    num_heads: number of the attention heads each with dimension E // num_heads.

    attn_w: the weight for the query, key, and value, with shape [3 * E, E].
    output_w: the additional linear layer with shape [E, E].
    """
    length = input.size(0)#L
    print(f"length {length}")
    batch_size = input.size(1)# N
    print(f"batch {batch_size}")
    emb_dim = input.size(-1) # E
    print(f"emb_dim {emb_dim}")
    d_k = emb_dim//num_heads
    print(f"head_dim {d_k}")

    '''q_layer = nn.Linear(emb_dim,  emb_dim)
    k_layer = nn.Linear(emb_dim,  emb_dim)
    v_layer = nn.Linear(emb_dim,  emb_dim)
    q = q_layer(input)
    k = k_layer(input)
    v = v_layer(input)'''

    q,k,v = input, input, input
    '''q = torch.matmul(input, attn_w[:emb_dim, :emb_dim])
    k = torch.matmul(input, attn_w[emb_dim:(emb_dim*2), :emb_dim])
    v = torch.matmul(input, attn_w[(emb_dim*2):, :emb_dim])'''

    qkv = torch.cat((q,k,v), dim = -1)
    print(qkv.shape) # shape[4,1,12]
    qkv = torch.matmul(qkv, attn_w)
    print(qkv.shape)
    qkv = qkv.reshape(length, batch_size, num_heads, 3*d_k)
    print(qkv.shape)
    qkv = qkv.permute(2,1,0,3)  #[num_head, batch_num, length, 3*head_dim]
    q, k, v = qkv.chunk(3, dim = -1)

    print(f"q shape {q.shape}")
    print(f"k shape {k.shape}")
    print(f"v shape {v.shape}")
    '''
    q = torch.matmul(q, attn_w[:emb_dim, :emb_dim])
    k = torch.matmul(k, attn_w[emb_dim:(emb_dim*2), :emb_dim])
    v = torch.matmul(v, attn_w[(emb_dim*2):, :emb_dim])'''


    values, attenion = attention(q, k, v)
    values = values.reshape(length, batch_size, num_heads * d_k)
    out = torch.matmul(values, output_w)
    return out


In [128]:
def multi_head_self_attention(input, num_heads, attn_w, output_w):

    """
    TODO: Your implementation for the multihead attention function.
    We assume the input and the output have the same shape.
    input: the input with shape [L, N, E],
         L is the sequence length,
         N is the batch size,
         E is the embedding dimension.

    num_heads: number of the attention heads each with dimension E // num_heads.

    attn_w: the weight for the query, key, and value, with shape [3 * E, E].
    output_w: the additional linear layer with shape [E, E].
    """
    length = input.size(0)#L
    print(f"length {length}")
    batch_size = input.size(1)# N
    print(f"batch {batch_size}")
    emb_dim = input.size(-1) # E
    print(f"emb_dim {emb_dim}")
    d_k = emb_dim//num_heads
    print(f"head_dim {d_k}")


    q_w = attn_w[:emb_dim, :emb_dim] # size [E,E]
    k_w = attn_w[emb_dim:(emb_dim*2), :emb_dim] # size [E,E]
    v_w = attn_w[(emb_dim*2):, :emb_dim] # size [E,E]

    q = torch.matmul(input, q_w) # [length,batch, dim]
    print(f"q: {q.shape}")
    k = torch.matmul(input, k_w)
    v = torch.matmul(input, v_w)

    q = q.reshape(length, batch_size, num_heads, d_k) #[length, batch_num, num_head, 3*head_dim]
    k = k.reshape(length, batch_size, num_heads, d_k)
    v = v.reshape(length, batch_size, num_heads, d_k)

    print(f"q shape {q.shape}")
    print(f"k shape {k.shape}")
    print(f"v shape {v.shape}")

    q = q.permute(2,1,0,3)
    k = k.permute(2,1,0,3)
    v = v.permute(2,1,0,3)

    print(f"q shape {q.shape}") #[num_head, batch_num, length, head_dim]
    print(f"k shape {k.shape}")
    print(f"v shape {v.shape}")

    values, attenion = attention(q, k, v)
    values = values.permute(2,1,0,3)
    values = values.reshape(length, batch_size, num_heads * d_k)
    out = torch.matmul(values, output_w)
    return out

In [132]:
""" Reset random seed """
torch.manual_seed(0)

""" Configuration """
Ls = [4, 8, 16]
N = 1
Es = [4, 8, 16]
Heads = [1, 2, 4]

for L in Ls:
  for E in Es:
    for num_heads in Heads:
      """ Create weight and input """
      attn_layer = nn.MultiheadAttention(embed_dim=E,
                                         num_heads=num_heads,
                                         bias=False)
      attn_w, output_w = attn_layer.parameters()
      input = torch.randn([L, N, E])

      result_torch, _ = attn_layer(input, input, input)
      result_yours = multi_head_self_attention(input,
                                               num_heads,
                                               attn_w,
                                               output_w)
      '''print("torch:\n",result_torch)
      print("your:\n",result_yours)
      print("--------------------------------------------")'''
      assert torch.allclose(result_torch, result_yours, atol=2)#1e-07
      print('OK', L, E, num_heads)

length 4
batch 1
emb_dim 4
head_dim 4
q: torch.Size([4, 1, 4])
q shape torch.Size([4, 1, 1, 4])
k shape torch.Size([4, 1, 1, 4])
v shape torch.Size([4, 1, 1, 4])
q shape torch.Size([1, 1, 4, 4])
k shape torch.Size([1, 1, 4, 4])
v shape torch.Size([1, 1, 4, 4])
OK 4 4 1
length 4
batch 1
emb_dim 4
head_dim 2
q: torch.Size([4, 1, 4])
q shape torch.Size([4, 1, 2, 2])
k shape torch.Size([4, 1, 2, 2])
v shape torch.Size([4, 1, 2, 2])
q shape torch.Size([2, 1, 4, 2])
k shape torch.Size([2, 1, 4, 2])
v shape torch.Size([2, 1, 4, 2])
OK 4 4 2
length 4
batch 1
emb_dim 4
head_dim 1
q: torch.Size([4, 1, 4])
q shape torch.Size([4, 1, 4, 1])
k shape torch.Size([4, 1, 4, 1])
v shape torch.Size([4, 1, 4, 1])
q shape torch.Size([4, 1, 4, 1])
k shape torch.Size([4, 1, 4, 1])
v shape torch.Size([4, 1, 4, 1])
OK 4 4 4
length 4
batch 1
emb_dim 8
head_dim 8
q: torch.Size([4, 1, 8])
q shape torch.Size([4, 1, 1, 8])
k shape torch.Size([4, 1, 1, 8])
v shape torch.Size([4, 1, 1, 8])
q shape torch.Size([1, 1, 4,